In [ ]:
from postprocessing import *
from scipy import special
from tqdm import trange, tqdm
from multiprocessing import Pool
mpl.rcParams['figure.dpi']= 200

In [ ]:
density = 20
mach = 4
jetpower = 40
start = '15'

In [ ]:
simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
i_file = 20
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory_jet + filename, "r")

In [ ]:
x_voronoi = snap_data['PartType0/Coordinates'][:, 0]
y_voronoi = snap_data['PartType0/Coordinates'][:, 1]
z_voronoi = snap_data['PartType0/Coordinates'][:, 2]

v_x_voronoi = snap_data['PartType0/Velocities'][:, 0]
v_y_voronoi = snap_data['PartType0/Velocities'][:, 1]
v_z_voronoi = snap_data['PartType0/Velocities'][:, 2]

In [ ]:
# testing values

# v_x_voronoi = x_voronoi 
# v_y_voronoi = y_voronoi * 2
# v_z_voronoi = np.ones_like(x_voronoi)

In [ ]:
x_ = np.linspace(500, 1500, 51)
y_ = np.linspace(500, 1500, 51)
z_ = np.linspace(500, 1500, 51)

dx, dy, dz = np.diff(x_)[0], np.diff(y_)[0], np.diff(z_)[0]

# X, Y, Z = np.meshgrid(x_, y_, z_, indexing='ij')

In [ ]:
# dVx_dx = np.zeros((len(x_), len(y_), len(z_)))
# dVy_dy = np.zeros((len(x_), len(y_), len(z_)))
# dVz_dz = np.zeros((len(x_), len(y_), len(z_)))

In [ ]:
def d_dx(field, x, y, z, h):
    f_right2 = f(x + 2 * h, y, z, field)
    f_right1 = f(x + 1 * h, y, z, field)
    f_left2  = f(x - 2 * h, y, z, field)
    f_left1  = f(x - 1 * h, y, z, field)
    df_dx = (- f_right2 + 8 * f_right1 - 8 * f_left1 + f_left2) / 12 / h
    return df_dx

def d_dy(field, x, y, z, h):
    f_right2 = f(x, y + 2 * h, z, field)
    f_right1 = f(x, y + 1 * h, z, field)
    f_left2  = f(x, y - 2 * h, z, field)
    f_left1  = f(x, y - 1 * h, z, field)
    df_dy = (- f_right2 + 8 * f_right1 - 8 * f_left1 + f_left2) / 12 / h
    return df_dy

def d_dz(field, x, y, z, h):
    f_right2 = f(x, y, z + 2 * h, field)
    f_right1 = f(x, y, z + 1 * h, field)
    f_left2  = f(x, y, z - 2 * h, field)
    f_left1  = f(x, y, z - 1 * h, field)
    df_dz = (- f_right2 + 8 * f_right1 - 8 * f_left1 + f_left2) / 12 / h
    return df_dz

In [ ]:
def f(x, y, z, field='v_x'):
    index = np.argmin((x_voronoi - x) ** 2 + (y_voronoi - y) ** 2 + (z_voronoi - z) ** 2)
    if field == 'v_x':
        return v_x_voronoi[index]
    if field == 'v_y':
        return v_y_voronoi[index]
    if field == 'v_z':
        return v_z_voronoi[index]

In [ ]:
# for x_i in trange(len(x_)):
#     for y_i in range(len(y_)):
#         for z_i in range(len(z_)):
#             dVx_dx[x_i, y_i, z_i] = d_dx('v_x', x_[x_i], y_[y_i], z_[z_i], dx)

$$r_{c s}=\frac{\left\langle|\nabla \cdot \vec{v}|^2\right\rangle}{\left\langle|\nabla \cdot \vec{v}|^2\right\rangle+\left\langle|\nabla \times \vec{v}|^2\right\rangle}$$

$$
\nabla \cdot \vec{v}=\frac{\partial v_x}{\partial x}+\frac{\partial v_y}{\partial y}+\frac{\partial v_z}{\partial z}
$$

$$\nabla \times \vec{v}=\left(\begin{array}{l}\frac{\partial v_z}{\partial y}-\frac{\partial v_y}{\partial z} \\ \frac{\partial v_x}{\partial z}-\frac{\partial v_z}{\partial x} \\ \frac{\partial v_y}{\partial x}-\frac{\partial v_x}{\partial y}\end{array}\right)$$

$$
\frac{\partial f}{\partial x}(x)=\frac{-f(x+2 h)+8 f(x+h)-8 f(x-h)+f(x-2 h)}{12 h}
$$

In [ ]:
## d/dx
def dvx_dx_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dx('v_x', x, y, z, dx)

def dvy_dx_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dx('v_y', x, y, z, dx)

def dvz_dx_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dx('v_z', x, y, z, dx)


## d/dy
def dvx_dy_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dy('v_x', x, y, z, dx)

def dvy_dy_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dy('v_y', x, y, z, dx)

def dvz_dy_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dy('v_z', x, y, z, dx)

## d/dz

def dvx_dz_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dz('v_x', x, y, z, dx)

def dvy_dz_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dz('v_y', x, y, z, dx)

def dvz_dz_wrapper(args):
    x, y, z, dx = args
    return x, y, z, d_dz('v_z', x, y, z, dx)


# Create a list of arguments for the d_dx_wrapper function
args_list = [(x_[x_i], y_[y_i], z_[z_i], dx) 
             for x_i in range(len(x_))
             for y_i in range(len(y_))
             for z_i in range(len(z_))]

# Use multiprocessing to parallelize the computation
with Pool() as pool:
    dvx_dx = np.array(list(tqdm(pool.imap(dvx_dx_wrapper, args_list), total=len(args_list))))
    dvy_dx = np.array(list(tqdm(pool.imap(dvy_dx_wrapper, args_list), total=len(args_list))))
    dvz_dx = np.array(list(tqdm(pool.imap(dvz_dx_wrapper, args_list), total=len(args_list))))
    
    dvx_dy = np.array(list(tqdm(pool.imap(dvx_dy_wrapper, args_list), total=len(args_list))))
    dvy_dy = np.array(list(tqdm(pool.imap(dvy_dy_wrapper, args_list), total=len(args_list))))
    dvz_dy = np.array(list(tqdm(pool.imap(dvz_dy_wrapper, args_list), total=len(args_list))))
    
    dvx_dz = np.array(list(tqdm(pool.imap(dvx_dz_wrapper, args_list), total=len(args_list))))
    dvy_dz = np.array(list(tqdm(pool.imap(dvy_dz_wrapper, args_list), total=len(args_list))))
    dvz_dz = np.array(list(tqdm(pool.imap(dvz_dz_wrapper, args_list), total=len(args_list))))

In [ ]:
if (np.array(dvx_dx)[:,0] == np.array(dvy_dy)[:,0]).all():
    if (np.array(dvx_dx)[:,0] == np.array(dvx_dz)[:,0]).all():
        if (np.array(dvy_dx)[:,1] == np.array(dvx_dz)[:,1]).all():
            if (np.array(dvx_dx)[:,1] == np.array(dvy_dz)[:,1]).all():
                if (np.array(dvz_dx)[:,2] == np.array(dvx_dz)[:,2]).all():
                    if (np.array(dvy_dx)[:,2] == np.array(dvy_dz)[:,2]).all():
                        div2 = (dvx_dx[:, 3] + dvy_dy[:, 3] + dvz_dz[:, 3]) ** 2
                        rot2 = (dvz_dy[:, 3] - dvy_dz[:, 3]) ** 2 + (dvx_dz[:, 3] - dvz_dx[:, 3]) ** 2 + (dvx_dy[:, 3] - dvy_dx[:, 3]) ** 2 
                        r_cs = np.mean(div2) / (np.mean(div2) + np.mean(rot2))

In [ ]:
# from time import sleep
# from random import random

# def task(arg):
#     # generate a value between 0 and 1
#     value = random()
#     # block for a fraction of a second to simulate work
#     sleep(value)
#     # # report the value to show progress
#     print(f'{arg} got {value}', flush=True)
 
# # entry point for the program
# if __name__ == '__main__':
#     # create the process pool
#     with Pool(10) as pool:
#         # call the same function with different data in parallel
#         pool.map(task, range(20))

In [ ]:
r_cs = 0.45

In [ ]:
np.savetxt(f'{output_directory_jet}/compressibility_{i_file}.txt', [r_cs])

In [ ]:
with open(f'{output_directory_jet}/compressibility_{i_file}.txt', 'rb') as f:
    a = np.loadtxt(f)

In [ ]:
a

In [ ]:
jet_rc  = np.array([[15.1025,  2.676619905274570388e-01], [15.7031, 2.190456090752272156e-01], [17.71, 1.177561085786484607e-01]])
turb_rc = np.array([[15.1025, 2.676619905274570388e-01], [15.7031, 2.579235102134883051e-01], [17.71, 2.750051801978366517e-01]])

In [ ]:
plt.plot(jet_rc.T[0] * unit_time_in_megayr, jet_rc.T[1], marker='o', label=r'jet on $10^{40}$ erg/s')
plt.plot(turb_rc.T[0] * unit_time_in_megayr, turb_rc.T[1], marker='o', label='turb box')
plt.axvline(15, c='gray', ls='--')
plt.ylabel(r'$r_{cs}$')
plt.xlabel('time, Myr')
plt.legend(loc='lower left')